In [82]:
import requests
import csv
import pandas as pd
import numpy as np

query = """
[out:json][timeout:180];
area["name"="Москва"]["boundary"="administrative"]->.moscow;
(
  way["shop"="mall"](area.moscow);
  relation["shop"="mall"](area.moscow);
  way["mall"="yes"](area.moscow);
  relation["mall"="yes"](area.moscow);
);
out tags center;
"""

r = requests.post("https://overpass-api.de/api/interpreter", data=query)
data = r.json()["elements"]

with open("moscow_malls.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["name", "lat", "lon", "osm_id"])
    for obj in data:
        tags = obj.get("tags", {})
        name = tags.get("name", "")
        lat = obj.get("center", {}).get("lat")
        lon = obj.get("center", {}).get("lon")
        writer.writerow([name, lat, lon, obj["id"]])


In [83]:
import requests
import csv

query = """
[out:json][timeout:180];
area["name"="Москва"]["boundary"="administrative"]->.moscow;
(
  node["amenity"="school"](area.moscow);
  way["amenity"="school"](area.moscow);
  relation["amenity"="school"](area.moscow);

  node["amenity"="university"](area.moscow);
  way["amenity"="university"](area.moscow);
  relation["amenity"="university"](area.moscow);
);
out center;
"""

r = requests.post("https://overpass-api.de/api/interpreter", data=query)
data = r.json()["elements"]

with open("moscow_schools_universities.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["name", "type", "lat", "lon", "osm_id"])
    for obj in data:
        tags = obj.get("tags", {})
        name = tags.get("name", "")
        kind = tags.get("amenity", "")

        lat = obj.get("lat") or obj.get("center", {}).get("lat")
        lon = obj.get("lon") or obj.get("center", {}).get("lon")

        writer.writerow([name, kind, lat, lon, obj["id"]])


In [85]:
unis = pd.read_csv("moscow_schools_universities.csv")

In [86]:
del unis['osm_id']

In [87]:
unis.dropna(inplace=True)

In [88]:
malls = pd.read_csv("moscow_malls.csv")

In [89]:
del malls['osm_id']

In [90]:
malls.dropna(inplace=True)

In [91]:
malls.insert(1, "type", "mall")

In [92]:
malls

,name,type,lat,lon
0,"Торговый центр ""Капитолий""",mall,55.880929,37.449631
1,"Торговый центр ""Александр Лэнд""",mall,55.885596,37.602184
2,Звёздный,mall,55.676925,37.504814
3,ЕвроПарк,mall,55.766862,37.381589
4,Место встречи Байконур,mall,55.864886,37.605004
...,...,...,...,...
494,Торгово-офисный центр «Аркадия»,mall,55.744568,37.630159
495,СпортEX,mall,55.737733,37.716609
497,Звёздочка,mall,55.741789,37.655813
498,Botanica,mall,55.844268,37.637250


In [93]:
df = pd.concat([malls, unis])

In [95]:
df.to_csv("POI.csv")